# Decision Tree
https://www.youtube.com/watch?v=NxEHSAfFlK8 <br>
Need to decide on
- Which feature do we split on
- Where to split (eg with numerical values)
- When to stop


<h3>Taking a sample from original dataset</h3>
- DO NOT RERUN THIS CELL! THE SAMPLE DATASET WAS EXPORTED AS "SAMPLE.CSV"
- only work on this sample during development
TY <3

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('diabetes_multiclass.csv') 
_, sample_df = train_test_split(df, test_size=5000, stratify=df["Diabetes_012"], random_state=42)

sample_df.to_csv('sample.csv', index=False)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
class Node:
    """
    A class made for nodes of a decision tree
    """
    # to pass a value to the var 'value', you must call the name (because of *)
    def __init__(self, feature=None, threshold=None, left=None, right=None, *,value=None):
        # Which feature to split on
        self.feature = feature
        # Which threshold to split on (where)
        self.threshold = threshold
        # Left node we're pointing to
        self.left = left
        # Right node we're pointing to
        self.right = right
        # Value of the node. Incase it is not a leaf node, value is None
        self.value = value
        
    def is_leaf_node(self):
        """
        This method checks whether a node is a leaf node or not

        Returns:
        True if the node is a leaf node
        False id the node is NOT a leaf node
        """
        return self.value is not None


class DecisionTree:
    """
    A class for a decision tree
    """
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        # Stopping criteria
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        # Add randomness to tree by eg using subset of features
        self.n_features=n_features
        # Root, start of tree
        self.root=None

    def fit(self, X, y):
        """
        This method initializes the number of features and grows the decision tree model using the input data.
        It assigns the root node of the decision tree model to the instance variable `self.root`.        
        """
        # Making sure that parameter n_features does not exceed self.n_features
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        # Building tree recursively
        self.root = self._grow_tree(X, y)

    # Helper method
    def _grow_tree(self, X, y, depth=0):
        """
        method that grows the tree recursively, helper method used in fit method

        Return:
        Node with the best feature, best threshold, the left tree and right tree
        """
        n_samples, n_featu = X.shape
        # Get all unique labels (values of y)
        n_labels = len(np.unique(y))

        # Check the stopping criteria
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value = leaf_value)
        
        # We create the randomness of our decision tree here, no duplicates
        feature_indices = np.random.choice(n_featu, self.n_features, replace=False)
        # Find the best split
        best_feature, best_threshold = self._best_split(X, y, feature_indices)

        # Create child nodes, create new subtrees
        left_indices, right_indices = self._split(X[:, best_feature], best_threshold)
        # Depth increase by 1 as we increase child nodes
        left_tree = self._grow_tree(X[left_indices, :], y[left_indices], depth+1)
        right_tree = self._grow_tree(X[right_indices, :], y[right_indices], depth+1)

        return Node(best_feature, best_threshold, left_tree, right_tree)

    # Helper method to select best split
    def _best_split(self, X, y, feat_idexes):
        """
        Helper method that calculates the best possible split based on the information gain
        
        Return:
        Split place and split threshold
        """
        # Initial set as -1
        best_gain = -1
        split_idex, split_threshold = None, None

        for feat_idex in feat_idexes:
            # Get column
            X_column = X[:, feat_idex]
            # Get only the unique ones
            thresholds = np.unique(X_column)

            for threshold in thresholds:
                # Calculate the information gain
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idex = feat_idex
                    split_threshold = threshold

        return split_idex, split_threshold

    # Helper method for information gain (IG)
    def _information_gain(self, y, X_column, threshold):
        """
        Helper method that calculates the information gain

        Return:
        Information gain
        """
        # Parent entropy
        parent_entropy = self._entropy(y)

        # Create children
        left_idices, right_idices = self._split(X_column, threshold)

        if len(left_idices) == 0 or len(right_idices) == 0:
            return 0
        
        # Calculate the weighted average entropy of the children
        n = len(y)
        n_left, n_right = len(left_idices), len(right_idices)
        entropy_left, entropy_right = self._entropy(y[left_idices]), self._entropy(y[right_idices])
        child_entropy = (n_left/n) * entropy_left + (n_right/n) * entropy_right

        # Calculate the IG
        information_gain = parent_entropy - child_entropy

        return information_gain

    # Helper method for splitting the tree
    def _split(self, X_column, split_thresh):
        """
        Helper method that splits the tree creating children (left and right)

        Return:
        Left and right indices of the subtrees/children
        """
        # Gives you one list of values (.flatten())
        left_indices = np.argwhere(X_column <= split_thresh).flatten()
        right_indices = np.argwhere(X_column > split_thresh).flatten()
        
        return left_indices, right_indices

    # Helper method to calculate the entropy for the information gain
    def _entropy(self, y):
        """
        Helper method that calculates the entropy

        Return:
        Entropy
        """
        # Count the occurances in bins (histogram)
        hist = np.bincount(y)
        ps = hist / len(y)
        # Entropy formula
        entropy = -np.sum([p * np.log2(p) for p in ps if p>0])

        return entropy

    # Helper function to calculate the value of y (label)
    def _most_common_label(self, y):
        """
        Helper method used in the _grow_tree_method to calculate the value of a node
        
        Return:
        Returns the value of y
        """
        # Get most common label's tuple and the first info including the value
        labels, counts = np.unique(y, return_counts=True)
        max_count_idx = np.argmax(counts)
        value = labels[max_count_idx]

        return value

    def predict(self, X):
        """
        method that traverse X and find the results of each x in X

        Return:
        A list containing all the values of x
        """
        # Find result of every x and return it
        predict_values = np.array([self._traverse_tree(x, self.root) for x in X])

        return predict_values
    
    # Helper method that traverses the tree
    def _traverse_tree(self, x, node):
        """
        Recursive method that traverses the tree until a leaf node is reached

        Return:
        Node value (if it's a leaf node)
        """
        # Check that node is a leaf node
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            # Traverse left tree
            return self._traverse_tree(x, node.left)
        # Otherwise traverse right tree
        return self._traverse_tree(x, node.right)
        

# Random forest
https://youtu.be/kFwe2ZZU7yw?si=GUzGBv--EWfy6_Rs

In [3]:
class RandomForest:
    """
    A class for the random forest algorithm 
    """
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, n_feature=None):
        # Amount of trees we want
        self.n_trees = n_trees
        # Stop conditions
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        # Add randomness to tree by eg using subset of features
        self.n_features = n_feature
        # Where the trees will be stored
        self.trees = []

    def fit(self, X, y):
        """
        Build a random forest by fitting decision trees to random subsets of the data.
        """
        # Initialize empty list
        self.trees = []
        # Create decision tree for every tree (based on the parameter n_trees)
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth,
                            min_samples_split=self.min_samples_split,
                            n_features=self.n_features)
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            # Add tree to list
            self.trees.append(tree)

    # Helper method for samples
    def _bootstrap_samples(self, X, y):
        """
        Create a bootstrapped sample of the data by randomly selecting samples with replacement
        """
        # First get number of samples
        n_samples = X.shape[0]
        # A used sample can be used again
        idices = np.random.choice(n_samples, n_samples, replace=True)

        return X[idices], y[idices]

    # Training part done, now predict

    # Helper method to calculate value of y
    def _most_common_label(self, y):
        """
        Helper method used in the predict method to calculate the value of a node
        
        Return:
        Returns the value of y (label)
        """
        # Get most common label's tuple and the first info including the value
        labels, counts = np.unique(y, return_counts=True)
        max_count_idx = np.argmax(counts)
        value = labels[max_count_idx]

        return value

    def predict(self, X):
        """
        Predict class labels for input samples. 
        Iterates over each decision tree in the forest and collects predictions for each sample.
        The final prediction for each sample is determined by selecting the most common label across all decision trees.
        """
        # Contains a list with a list containing the samples eg [[1,2,3], [1,3,2]]
        predictions = np.array([tree.predict(X) for tree in self.trees])
        # Get predictions from same sample of different tree in one list: 
        # first sample from first tree and first sample from second tree will be in the first list
        tree_preds = np.swapaxes(predictions, 0, 1)
        # Get list with predictions of most common label
        predictions = np.array([self._most_common_label(pred) for pred in tree_preds])

        return predictions

In [4]:
# Added function to make randomforest work with sklearn
def read_data_and_return_arrays(file_path, target_column):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Extract features
    features = df.drop(columns=[target_column])
    data = features.values
    
    # Extract target variable
    target = df[target_column].astype(int).values
    
    # Extract target names if available
    target_names = None
    
    return data, target, target_names

In [5]:
data, target_rf, target_names = read_data_and_return_arrays(
            file_path="sample.csv", target_column='Diabetes_012'
        )
X = data
y = target_rf

x_train_rf, x_test_rf, y_train_rf, y_test_rf = train_test_split(X, y, test_size=0.2, random_state=92)

random_forest = RandomForest()
random_forest.fit(x_train_rf, y_train_rf)
rf_predictions = random_forest.predict(x_test_rf)
rf_score = accuracy_score(y_test_rf, rf_predictions)
print('predictions:', rf_predictions)
print('accuracy:', rf_score)

predictions: [0 0 0 0 0 0 0 0 0 0 0 2 2 0 2 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0
 0 0 0 2 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0 0 2 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 2 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0
 0 0 0 0 0 2 0 0 2 2 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0
 0 0 0 0 0 0